In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Submission_data = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

In [ ]:
print((len(train)+1)/24, '> 24시간 시계열 중 비어있는 데이터가 존재 ' ) 
print((len(test)+1)/24, '> 24시간 시계열 중 비어있는 데이터가 존재 ' )

# train : 매월 1일 ~ 19일 
print(train['datetime'][0],'~', train['datetime'][10885])
#test : 매월 20일~ 말일 
print(test['datetime'][0],'~', test['datetime'][6492])

In [ ]:
print(train.columns)
print(test.columns) # casual, registered, count 빠져있음 

1. datetime : 시간. 연-월-일 시:분:초로 표현 (ex. 2018-11-01 00:00:00)

2. season : 계절. 봄(1), 여름(2), 가을(3), 겨울(4)

3. holiday : 공휴일. 공휴일(1), 공휴일X(0)

4. workingday : 근무일. 근무일(1), 근무일X(0)

5. weather : 날씨 

(1) 아주 깨끗한 날씨. 또는 아주 약간의 구름 (2) 약간의 안개와 구름이 끼어있는 날씨

(3) 약간의 눈, 비가 오거나 천둥 (4) 아주 많은 비가 오거나 우박

6. temp : 온도. 섭씨(Celsius)

7. atemp : 체감온도. 섭씨(Celsius)

8. humiduty : 습도

9. windspped : 풍속

10. casual : 비회원(non-resigtered)의 자전거 대여량

11. resistered : 회원(registered)의 자전거 대여량

12. count : 총 자전거 대여량. 비회원(casual) + 회원(resistered)와 동일

# EDA  

In [ ]:
train.describe()

In [ ]:
train.info()
# datetime > object

In [ ]:
train['datetime'] = pd.to_datetime(train['datetime'],format="%Y-%m-%d %H:%M:%S")
test['datetime'] = pd.to_datetime(test['datetime'],format="%Y-%m-%d %H:%M:%S")

In [ ]:
train

In [ ]:
train.isnull().sum()

In [ ]:
# 다중공선성 확인 
from matplotlib import pyplot as plt 
import seaborn as sns

cmap = sns.light_palette('darkgray', as_cmap=True)
sns.heatmap(train.corr(), annot=True, fmt='3.1f',cmap=cmap)
plt.show()

# casual과 count 가 높은 상관관계 > 변수간 관계가 아니므로 상관 x 
# temp 와 atemp > 상관관계 1 > 하나 제거(atemp 제거 )
#registerd, casual 0.5 상관관계  > 비회원과 회원의 관계이므로 두 변수 중 하나는 제거해야함 
# temp, casual 0.5 > 온도와 비회원 변수 그대로 둔다. 

In [ ]:
feature_name= ['season', 'holiday', 'workingday', 'weather', 'temp',
               'humidity', 'windspeed']

## 선형회귀모형

In [ ]:
train.head()

In [ ]:
import statsmodels.api as sm 

dfX0 = pd.DataFrame(train, columns=feature_name)
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(train, columns=['count'])

model = sm.OLS(dfy,dfX)
result = model.fit()

print(result.summary())

1. adj 알스퀘어값 0.259 > 설명력이 아주 낮음 
2. 변수 중 p > 0.05 : holiday, workingday, weather > 변수 필요 없 ... 왜일까 ?    날씨와 ... 쉬는날은 중요할거같은데 몬가 이상하다.
3. F_p < 0.05 모형자체는 유의미하다곤 볼 수 있따 ? .. 

# ARIMA

In [ ]:
# 시각화 
train['count'].plot()

# 증가하는 추세, 계절성 있다고 봐야하는가 ? 월별 데이터로 뽑아서 봐야함 
# 정상성 없음 > 차분 필요 


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
series=train['count']
plot_acf(series)
plot_pacf(series)
plt.show()

# p=5, q= 35 ? 